### Paxillin movie segmentation and object feature analysis

with Annabel's 3D fixed image

Liya Ding, 2024.03

In [11]:
import numpy as np
import seaborn as sns
import pandas as pd
import umap
from joblib import dump, load
import os.path

import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.transforms import Bbox

import tifffile 
import czifile

import skimage
from skimage.morphology import remove_small_objects
from skimage.measure import label, regionprops, regionprops_table
from skimage.color import label2rgb
from skimage.filters import (threshold_otsu, threshold_niblack,
                             threshold_sauvola)

from skimage.morphology import binary_opening, binary_dilation
from skimage.morphology import disk


from scipy import ndimage
from scipy.ndimage import distance_transform_cdt
from scipy.ndimage import gaussian_filter

from sklearn.decomposition import PCA
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# function from aicssegmentation tool
from utils.vessel_2d import filament_2d_wrapper
from utils.pre_processing_utils import intensity_normalization


In [12]:
czimovie_dir =  '/mnt/d/lding/FA/data/FHL2_Endothelial_c_endo_limbd1_phosphofak_actin_catalina_2020404'

csv_output_dir = '/mnt/d/lding/FA/analysis_results/fPak_catalina/Vess-LocThr-GlThr-Open_tab10_csv'
plot_output_dir = '/mnt/d/lding/FA/analysis_results/fPak_catalina/Vess-LocThr-GlThr-Open_tab10_plot'
seg_output_dir = '/mnt/d/lding/FA/analysis_results/fPak_catalina/Vess-LocThr-GlThr-Open_tab10_seg'

pixel_size = 0.0775
time_point = 0

if not os.path.isdir(csv_output_dir):
    os.makedirs(csv_output_dir)
if not os.path.isdir(plot_output_dir):
    os.makedirs(plot_output_dir)
if not os.path.isdir(seg_output_dir):
    os.makedirs(seg_output_dir)

In [13]:
# build the colormap with iterative tab10
tab10 = cm.get_cmap('tab10', 10)
for i in range(10):
    tab10.colors = np.concatenate([tab10.colors,tab10.colors],axis=0)    
tab10.colors = np.concatenate([np.zeros([1,4]),tab10.colors],axis=0)

newmap = cm.get_cmap('tab10', 4000+1)
newmap.colors = tab10.colors[0:4000+1,:]
newmap.colors[0,:]  = np.zeros([1,4])
newmap.colors[0,3] = 1

### for each 3D stack run through MIP and 8th frame

In [14]:
# flag for some optinal parts
flag_plot = True
flag_plot_save = True
flag_seg_save = True
flag_csv_save = True
flag_run_all = True
flag_panel_plot = True

In [17]:
### for each movie, make sub-folders for different output for easier viewing in imagej

movie_mip_output_dir = os.path.join(seg_output_dir,  'mip')
if not os.path.isdir(movie_mip_output_dir):   os.makedirs(movie_mip_output_dir)
   

filenames = [x for x in os.listdir(czimovie_dir) if os.path.isfile(os.path.join(czimovie_dir, x)) and ('.czi' in x)]

for filenameID in range(len(filenames)):
  
    filename = filenames[filenameID]
    img = czifile.imread(os.path.join(czimovie_dir,filename))
    pax_img = img[0,0,1,:,:,:,0].squeeze()
    
    MIP_pax_ori_img = pax_img.max(axis=0)
    tifffile.imsave(os.path.join(movie_mip_output_dir, 'SS_pax_img_'+filename+'_MIP'+'.tif'),MIP_pax_ori_img.astype(np.float32))   
  

/home/lding/anaconda3/envs/aics_segmentation/lib/python3.6/site-packages/ipykernel_launcher.py:112: FutureWarning: The new recommended value for bg_label is 0. Until version 0.19, the default bg_label value is -1. From version 0.19, the bg_label default value will be 0. To avoid this warning, please explicitly set bg_label value.
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping